## Setting up

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException,ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')  
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option(
        "prefs", {
            # block image loading
            "profile.managed_default_content_settings.images": 2,
        }
    )
# options.add_argument('--headless')  # Uncomment for headless mode

service = Service(ChromeDriverManager().install())

# driver = webdriver.Chrome(service=service, options=options)
# wait = WebDriverWait(driver, 10)
# driver.get("https://alonhadat.com.vn/-nha-gac-lung-mt-tan-thai-dt-75m2-gia-tot--14079635.html")


In [6]:
data_list = []
STANDARD_KEYS = ['price','area', 'Mã tin','Hướng','Phòng ăn', 'Loại tin','Đường trước nhà',
                 'Nhà bếp','Loại BDS', 'Pháp lý', 'Sân thượng', 'Chiều ngang', 'Số lầu',
                 'Chổ để xe hơi','Chiều dài','Số phòng ngủ','Chính chủ']

## Supporting function to avoid bot detection

In [7]:

def human_like_delay(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_like_delay(1, 3)


## Function to scrape the main page containg cards

In [8]:
def scrape_main_page(url):
    scroll_to_bottom()
    human_like_delay(2, 4)

    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'thumbnail')))

    thumbnails = driver.find_elements(By.XPATH, '//div[@class="thumbnail"]/a')

    for i in range(len(thumbnails)):
        try:
            print(f"Clicking on thumbnail {i + 1}...")
            
            driver.execute_script("arguments[0].scrollIntoView();", thumbnails[i])
            human_like_delay(1, 3)
            thumbnails[i].click() 

            human_like_delay(1, 3)

            scrape_page()
            
            human_like_delay(2, 5)

        except Exception as e:
            print(f"An error occurred while clicking on thumbnail {i + 1}: {e}")
        
        finally:
            # Go back to the main page after scraping
            driver.back()
            human_like_delay(2, 3)

## Lay data tu 1 card

In [9]:
def scrape_page():
    data_dict = {}
    human_like_delay(1,3)
    price = driver.find_element(By.XPATH, '//div[@class="moreinfor"]//span[@class="price"]//span[@class="value"]').text.strip()
    area = driver.find_element(By.XPATH, '//div[@class="moreinfor"]//span[@class="square"]//span[@class="value"]').text.strip()
    data_dict['price'] = price
    data_dict['area'] = area
    human_like_delay(2,4)
    info = driver.find_elements(By.XPATH, '//div[@class="moreinfor1"]//div[@class="infor"]/table/tbody/tr/td')
    for i, li in enumerate(info):
        try:
            if i % 2 == 0:
                key = li.text.strip()  
            else:
                try:
                    if li.find_element(By.TAG_NAME, 'img'):
                        val = 1
                except:
                    if li.text.strip() == '---':
                        val = None 
                    else:
                        val = li.text.strip() 
                
                data_dict[key] = val
        except (NoSuchElementException, StaleElementReferenceException) as e:
            print(f"Error processing element: {e}")
            continue

    for key in STANDARD_KEYS:
        if key not in data_dict:
            data_dict[key] = None
    data_list.append(data_dict)
    print(data_list)

In [13]:
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
try:
    for i in range(5,50):
        try:

            url = f'https://alonhadat.com.vn/can-ban-nha-da-nang-t3/trang-{i}.htm'
            card_links = scrape_main_page(url)
        except:
            break
except Exception:
    print("An error occurred while scraping the main page.")
finally:
    print(data_list)
    df = pd.DataFrame(data_list)
    df.to_csv('scraped_data_alonhadat.csv',encoding="utf-8", index=False)
    print("Saved successfully")
# driver.quit()


Collected link for card 1
Collected link for card 2
Collected link for card 3
Collected link for card 4
Collected link for card 5
Collected link for card 6
Collected link for card 7
Collected link for card 8
Collected link for card 9
Collected link for card 10
Collected link for card 11
Collected link for card 12
Collected link for card 13
Collected link for card 14
Collected link for card 15
Collected link for card 16
Collected link for card 17
Collected link for card 18
Collected link for card 19
Collected link for card 20
Scraping card 1
[{'price': '3,8 tỷ', 'area': '50 m\n2', 'Mã tin': '14692226', 'Hướng': 'Đông', 'Phòng ăn': None, 'Loại tin': 'Cần bán', 'Đường trước nhà': '7m', 'Nhà bếp': None, 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': 'Sổ hồng/ Sổ đỏ', 'Sân thượng': None, 'Chiều ngang': None, 'Số lầu': '2', 'Chổ để xe hơi': None, 'Chiều dài': None, 'Số phòng ngủ': '2', 'Chính chủ': None}, {'price': '9,7 tỷ', 'area': '100 m\n2', 'Mã tin': '15177465', 'Hướng': '_', 'Phòng ăn': 1, 'Loạ